In [ ]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 63.8 MB/s eta 0:00:00


#yolo keypoints

Unzip the folder into colab

In [ ]:
import zipfile
import os

zip_file_path = '/content/drive/MyDrive/Courses/פרוייקט/colab/yolo_kp.zip'
extract_dir = '/content' # Changed extract_dir to /content/

# Create the extraction directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"'{zip_file_path}' unzipped to '{extract_dir}'")

'/content/drive/MyDrive/Courses/פרוייקט/colab/yolo_kp.zip' unzipped to '/content'


Create a data yaml

In [ ]:
# 1) Make a tiny local YAML that points at /content/yolo_kp
yaml_text = """
path: /content/yolo_kp
train: images/train
val: images/val
test: images/test
names: [board]
kpt_shape: [4, 3]
"""
with open("/content/yolo_kp_local.yaml", "w") as f:
    f.write(yaml_text)
print(open("/content/yolo_kp_local.yaml").read())


path: /content/yolo_kp
train: images/train
val: images/val
test: images/test
names: [board]
kpt_shape: [4, 3]



Train

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8m-pose.pt")

run_name = "pose_kp_a100_medium_v1"
project_path = "/content/drive/MyDrive/Courses/פרוייקט/colab/runs"

print(f"Starting training run: {run_name}")
print(f"Results will be saved to: {project_path}")

# --- 3. Start Training ---
results = model.train(
    data="/content/yolo_kp_local.yaml",
    epochs=80,
    patience=12,          # Keep early stopping
    imgsz=1024,

    # --- A100 GPU Optimizations ---
    batch=32,             # Increased batch size for A100's 40GB VRAM (from 8)
    workers=8,            # Increased dataloader workers for the faster GPU (from 2)
    amp=True,             # Enable Automatic Mixed Precision for A100 Tensor Cores

    # --- Model Improvement Suggestions ---
    augment=True,         # Enable data augmentation to improve generalization
    cos_lr=True,          # Use cosine learning rate schedule for better convergence

    # --- Original Parameters ---
    device=0,
    cache=False,          # Keep False as data is in /content
    pretrained=True,
    project=project_path, # Use the project_path variable
    name=run_name         # Use the run_name variable
)

print(f"Training finished.")
print(f"Best model saved to: {project_path}/{run_name}/weights/best.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Starting training run: pose_kp_a100_medium_v1
Results will be saved to: /content/drive/MyDrive/Courses/פרוייקט/colab/runs
Ultralytics 8.3.221 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/yolo_kp_local.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=80, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, form

Evaluation

In [ ]:
from ultralytics import YOLO
import os

# --- 1. Define Paths ---
project_path = "/content/drive/MyDrive/Courses/פרוייקט/colab/runs"

# Path to your OLD model's weights
model_old_name = "pose_2k8"
model_old_path = os.path.join(project_path, model_old_name, "weights/best.pt")

# Path to your NEW model's weights (the one we just trained)
model_new_name = "pose_kp_a100_medium_v1"
model_new_path = os.path.join(project_path, model_new_name, "weights/best.pt")

# Path to your data configuration file
data_yaml_path = "/content/yolo_kp_local.yaml"

print(f"Comparing models:")
print(f"OLD: {model_old_path}")
print(f"NEW: {model_new_path}")

# --- 2. Load Models ---
try:
    model_old = YOLO(model_old_path)
    print("\nSuccessfully loaded OLD model (pose_2k8).")
except Exception as e:
    print(f"Error loading OLD model: {e}")

try:
    model_new = YOLO(model_new_path)
    print("Successfully loaded NEW model (pose_kp_a100_medium_v1).")
except Exception as e:
    print(f"Error loading NEW model: {e}")

# --- 3. Evaluate OLD Model ---
print("\n--- Evaluating OLD model (pose2k8) ---")
# We run val() and give it a unique name to save the results
metrics_old = model_old.val(
    data=data_yaml_path,
    imgsz=1024,
    batch=32,
    device=0,
    name=f"eval_{model_old_name}"
)
print("------------------------------------------")

# --- 4. Evaluate NEW Model ---
print("\n--- Evaluating NEW model (pose_kp_a100_medium_v1) ---")
# We run val() on the same data for a fair comparison
metrics_new = model_new.val(
    data=data_yaml_path,
    imgsz=1024,
    batch=32,
    device=0,
    name=f"eval_{model_new_name}"
)
print("------------------------------------------")

# --- 5. Print Comparison Summary ---
# The 'metrics' object contains all the results.
# We are most interested in the keypoint (pose) metrics.
print("\n\n--- Final Comparison Summary (Higher is Better) ---")

print("\nPose mAP50-95 (Primary Metric):")
print(f"  OLD (pose_2k8):                 {metrics_old.pose.map:.4f}")
print(f"  NEW (pose_kp_a100_medium_v1):  {metrics_new.pose.map:.4f}")

print("\nPose mAP50 (Easy Metric):")
print(f"  OLD (pose_2k8):                 {metrics_old.pose.map50:.4f}")
print(f"  NEW (pose_kp_a100_medium_v1):  {metrics_new.pose.map50:.4f}")

print("\nBounding Box mAP50-95 (How well it found the board):")
print(f"  OLD (pose_2k8):                 {metrics_old.box.map:.4f}")
print(f"  NEW (pose_kp_a100_medium_v1):  {metrics_new.box.map:.4f}")

print("\n\nEvaluation complete. You can see the full results (like PR curves)")
print(f"in the folders: {project_path}/eval_{model_old_name}")
print(f"and: {project_path}/eval_{model_new_name}")

Comparing models:
OLD: /content/drive/MyDrive/Courses/פרוייקט/colab/runs/pose_2k8/weights/best.pt
NEW: /content/drive/MyDrive/Courses/פרוייקט/colab/runs/pose_kp_a100_medium_v1/weights/best.pt

Successfully loaded OLD model (pose2k8).
Successfully loaded NEW model (pose_kp_a100_medium_v1).

--- Evaluating OLD model (pose2k8) ---
Ultralytics 8.3.221 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLOv8s-pose summary (fused): 81 layers, 11,413,047 parameters, 0 gradients, 29.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4500.2±863.8 MB/s, size: 2482.1 KB)
val: Scanning /content/yolo_kp/labels/val.cache... 330 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 330/330 706.9Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 11/11 1.2it/s 9.1s
                   all        330        330          1          1      0.995       0.99      0.9

speed test

In [ ]:
import numpy as np
import random
import glob
import os
import time
from ultralytics import YOLO

# --- 1. Define Paths ---
# Path to the validation images for your KEYPOINT model
VAL_DIR = "/content/yolo_kp/images/val"

# Path to your NEW model's weights
model_new_path = "/content/drive/MyDrive/Courses/פרוייקט/colab/runs/pose_kp_a100_medium_v1/weights/best.pt"

print(f"Loading model for speed test: {model_new_path}")
print(f"Using validation images from: {VAL_DIR}")

# --- 2. Load Images ---
# Get all jpg and png images from the val directory
imgs = glob.glob(os.path.join(VAL_DIR, "*.jpg")) + glob.glob(os.path.join(VAL_DIR, "*.png"))

# Create a subset of 100 images to test on (or all if fewer than 100)
if len(imgs) > 100:
    random.seed(0) # Use a fixed seed for reproducible results
    subset = random.sample(imgs, 100)
else:
    subset = imgs
print(f"Found {len(imgs)} images, testing on {len(subset)} images.")

# --- 3. Load Model ---
model = YOLO(model_new_path)

# --- 4. Run Speed Test ---
# Run prediction once to "warm up" the GPU
print("Warming up GPU...")
model.predict(source=subset[0], imgsz=1024, verbose=False)

print("Starting speed test...")
t0 = time.time()
# Run inference on the full subset
# We set verbose=False to not print results for every single image
pred = model.predict(
    source=subset,
    imgsz=1024,
    verbose=False,
    device=0
)
dt = (time.time() - t0)

# --- 5. Calculate and Print Results ---
ms_per_img = (dt / len(subset)) * 1000
fps = 1000 / ms_per_img

print("\n--- Speed Test Complete ---")
print(f"Processed {len(subset)} images in {dt:.3f} seconds.")
print(f"Average speed: {ms_per_img:.1f} ms/img")
print(f"Frames Per Second (FPS): {fps:.1f}")

Loading model for speed test: /content/drive/MyDrive/Courses/פרוייקט/colab/runs/pose_kp_a100_medium_v1/weights/best.pt
Using validation images from: /content/yolo_kp/images/val
Found 330 images, testing on 100 images.
Warming up GPU...
Starting speed test...

--- Speed Test Complete ---
Processed 100 images in 17.610 seconds.
Average speed: 176.1 ms/img
Frames Per Second (FPS): 5.7


evaluate on the test set

In [ ]:
from ultralytics import YOLO
import os

# --- 1. Define Paths ---
project_path = "/content/drive/MyDrive/Courses/פרוייקט/colab/runs"
data_yaml_path = "/content/yolo_kp_local.yaml" # Use the keypoint yaml

# --- 2. Load your BEST keypoint model ---
model_name = "pose_kp_a100_medium_v1"
model_path = os.path.join(project_path, model_name, "weights/best.pt")

print(f"Loading FINAL keypoint model for testing: {model_path}")

model = YOLO(model_path)

# --- 3. Run evaluation on the TEST set ---
print("\n--- Running on Keypoint TEST Set ---")
metrics_kp_test = model.val(
    data=data_yaml_path,
    imgsz=1024,
    batch=32,
    device=0,
    name=f"FINAL_TEST_{model_name}",
    split='test'  # <-- This is the important part
)

print("\n--- Keypoint FINAL TEST Results ---")
print(f"Pose mAP50-95 (Final Score): {metrics_kp_test.pose.map:.4f}")

Loading FINAL keypoint model for testing: /content/drive/MyDrive/Courses/פרוייקט/colab/runs/pose_kp_a100_medium_v1/weights/best.pt

--- Running on Keypoint TEST Set ---
Ultralytics 8.3.221 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLOv8m-pose summary (fused): 101 layers, 26,402,263 parameters, 0 gradients, 80.8 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3185.3±355.8 MB/s, size: 2418.2 KB)
val: Scanning /content/yolo_kp/labels/test.cache... 306 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 306/306 707.1Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 10/10 1.2it/s 8.3s
                   all        306        306          1          1      0.995      0.989          1          1      0.995      0.995
Speed: 4.0ms preprocess, 5.8ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to /content/runs/pose/FINAL

#yolo piece recognition

Unzip the folder into colab

In [ ]:
import zipfile
import os

zip_file_path = '/content/drive/MyDrive/Courses/פרוייקט/colab/yolo_det.zip'
extract_dir = '/content' # Changed extract_dir to /content/

# Create the extraction directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"'{zip_file_path}' unzipped to '{extract_dir}'")

'/content/drive/MyDrive/Courses/פרוייקט/colab/yolo_det.zip' unzipped to '/content'


Create a data yaml

In [ ]:
DATA_DIR = "/content/yolo_det"

data_yaml = f"""
path: {DATA_DIR}
train: images/train
val: images/val
test: images/test
names:
  0: white-king
  1: white-queen
  2: white-rook
  3: white-bishop
  4: white-knight
  5: white-pawn
  6: black-king
  7: black-queen
  8: black-rook
  9: black-bishop
  10: black-knight
  11: black-pawn
"""
with open(f"{DATA_DIR}/data.yaml", "w") as f:
    f.write(data_yaml)

print(open(f"{DATA_DIR}/data.yaml").read())



path: /content/yolo_det
train: images/train
val: images/val
test: images/test
names:
  0: white-king
  1: white-queen
  2: white-rook
  3: white-bishop
  4: white-knight
  5: white-pawn
  6: black-king
  7: black-queen
  8: black-rook
  9: black-bishop
  10: black-knight
  11: black-pawn



Train

In [ ]:
from ultralytics import YOLO
import os, torch

# Ensure GPU is set up
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True,max_split_size_mb:128"
torch.backends.cudnn.benchmark = True

DATA = "/content/yolo_det/data.yaml"
MODEL = "yolov8m.pt"   # 'm' is a great start. We can try 'l' if needed.

# Give this new run a clear name
run_name = "det_pieces_a100_v1_augmented"
project_path = "/content/drive/MyDrive/Courses/פרוייקט/colab/runs"

print(f"Starting main detection training: {run_name}")

model = YOLO(MODEL)
results = model.train(
    data=DATA,
    imgsz=1024,
    epochs=100,
    patience=20,

    # --- KEY CHANGES ---
    augment=True,         # *** This is the most important change ***
    workers=8,            # A100 Optimization (was 2)

    # --- Good original settings ---
    batch=32,
    device=0,
    amp=True,
    cos_lr=True,
    cache=False,
    project=project_path,
    name=run_name
)

print(f"\nMain training complete.")
print(f"Best model saved to: {project_path}/{run_name}/weights/best.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Starting main detection training: det_pieces_a100_v1_augmented
Ultralytics 8.3.221 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/yolo_det/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv

Polish and fine tune

In [ ]:
from ultralytics import YOLO
import os

# --- 1. Define Paths ---
DATA = "/content/yolo_det/data.yaml"
project_path = "/content/drive/MyDrive/Courses/פרוייקט/colab/runs"

# --- 2. Define Run Names ---
# This is the run that JUST finished and stopped early
main_run_name = "det_pieces_a100_v1_augmented"

# This will be the name of our new polish run
polish_run_name = "det_pieces_a100_v1_polish"

# --- 3. Set Path to the Best Model ---
# This is the best.pt file from epoch 12 that the last run saved
BEST_WEIGHTS = os.path.join(project_path, main_run_name, "weights/best.pt")

print(f"Loading weights for polishing: {BEST_WEIGHTS}")

# --- 4. Load the Model ---
model = YOLO(BEST_WEIGHTS)

# --- 5. Start Polish Training ---
print(f"Starting polish run: {polish_run_name}")

results_polish = model.train(
    data=DATA,
    epochs=20,            # A short 20-epoch polish run
    lr0=1e-4,             # Very low learning rate for stable fine-tuning
    imgsz=1024,
    batch=32,             # Keep large batch
    workers=8,
    amp=True,
    patience=0,           # No early stopping for a short polish run

    # --- Disable heavy augmentations for polishing ---
    close_mosaic=20,      # Disable mosaic for all 20 epochs
    augment=False,        # Turn off other augmentations (like flips, hsv)

    device=0,
    cache=False,
    project=project_path, # Save to our main project folder
    name=polish_run_name  # Save in its own new folder
)

print(f"\nPolish run complete.")
print(f"Final polished model saved to: {project_path}/{polish_run_name}/weights/best.pt")

Loading weights for polishing: /content/drive/MyDrive/Courses/פרוייקט/colab/runs/det_pieces_a100_v1_augmented/weights/best.pt
Starting polish run: det_pieces_a100_v1_polish
Ultralytics 8.3.221 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=20, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/yolo_det/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/drive/My

Evaluate the 3 versions

In [ ]:
from ultralytics import YOLO
import os

# --- 1. Define Paths ---
project_path = "/content/drive/MyDrive/Courses/פרוייקט/colab/runs"
data_yaml_path = "/content/yolo_det/data.yaml"

# --- 2. Define Model Names ---

# --- Model 1: Your original "safe" model ---
model_old_name = "det_pieces_a100_safe"
model_old_path = os.path.join(project_path, model_old_name, "weights/best.pt")

# --- Model 2: The new "augmented" model (pre-polish) ---
model_new_aug_name = "det_pieces_a100_v1_augmented"
model_new_aug_path = os.path.join(project_path, model_new_aug_name, "weights/best.pt")

# --- Model 3: The new "polished" model (what is running now) ---
model_new_pol_name = "det_pieces_a100_v1_polish"
model_new_pol_path = os.path.join(project_path, model_new_pol_name, "weights/best.pt")

print("Starting evaluation for 3 models...")
print(f"  OLD (safe):      {model_old_path}")
print(f"  NEW (Augmented): {model_new_aug_path}")
print(f"  NEW (Polished):  {model_new_pol_path}")

# --- 3. Load Models ---
try:
    model_old = YOLO(model_old_path)
    model_new_aug = YOLO(model_new_aug_path)
    model_new_pol = YOLO(model_new_pol_path)
    print("\nAll models loaded successfully.")
except Exception as e:
    print(f"\nError loading one or more models: {e}")
    print("Please check the model names and paths above.")
    # We will stop here if a model fails to load
    raise e

# --- 4. Evaluate Each Model ---
print(f"\n--- Evaluating OLD model ({model_old_name}) ---")
metrics_old = model_old.val(
    data=data_yaml_path,
    imgsz=1024,
    batch=32,
    device=0,
    name=f"eval_{model_old_name}"
)

print(f"\n--- Evaluating NEW Augmented model ({model_new_aug_name}) ---")
metrics_new_aug = model_new_aug.val(
    data=data_yaml_path,
    imgsz=1024,
    batch=32,
    device=0,
    name=f"eval_{model_new_aug_name}"
)

print(f"\n--- Evaluating NEW Polished model ({model_new_pol_name}) ---")
metrics_new_pol = model_new_pol.val(
    data=data_yaml_path,
    imgsz=1024,
    batch=32,
    device=0,
    name=f"eval_{model_new_pol_name}"
)

# --- 5. Print Final Comparison ---
print("\n\n--- Final Comparison Summary (Higher is Better) ---")

print("\nmAP50-95 (Primary Metric):")
print(f"  OLD ({model_old_name}):           {metrics_old.box.map:.4f}")
print(f"  NEW Aug. ({model_new_aug_name}):  {metrics_new_aug.box.map:.4f}")
print(f"  NEW Polish ({model_new_pol_name}): {metrics_new_pol.box.map:.4f}")

print("\nmAP50 (Easy Metric):")
print(f"  OLD ({model_old_name}):           {metrics_old.box.map50:.4f}")
print(f"  NEW Aug. ({model_new_aug_name}):  {metrics_new_aug.box.map50:.4f}")
print(f"  NEW Polish ({model_new_pol_name}): {metrics_new_pol.box.map50:.4f}")

Starting evaluation for 3 models...
  OLD (safe):      /content/drive/MyDrive/Courses/פרוייקט/colab/runs/det_pieces_a100_safe/weights/best.pt
  NEW (Augmented): /content/drive/MyDrive/Courses/פרוייקט/colab/runs/det_pieces_a100_v1_augmented/weights/best.pt
  NEW (Polished):  /content/drive/MyDrive/Courses/פרוייקט/colab/runs/det_pieces_a100_v1_polish/weights/best.pt

All models loaded successfully.

--- Evaluating OLD model (det_pieces_a100_safe) ---
Ultralytics 8.3.221 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 92 layers, 25,846,708 parameters, 0 gradients, 78.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 5165.6±1163.2 MB/s, size: 2318.9 KB)
val: Scanning /content/yolo_det/labels/val.cache... 330 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 330/330 613.8Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 11/11 0.6it/s 19.0s
                   all

speed check

In [ ]:
import numpy as np
import random
import glob
import os
import time
from ultralytics import YOLO

# --- 1. Define Paths ---
# Path to the validation images for your PIECE DETECTION model
VAL_DIR = "/content/yolo_det/images/val"

# Path to your NEW, POLISHED model's weights
model_best_path = "/content/drive/MyDrive/Courses/פרוייקט/colab/runs/det_pieces_a100_v1_polish/weights/best.pt"

print(f"Loading model for speed test: {model_best_path}")
print(f"Using validation images from: {VAL_DIR}")

# --- 2. Load Images ---
imgs = glob.glob(os.path.join(VAL_DIR, "*.jpg")) + glob.glob(os.path.join(VAL_DIR, "*.png"))

# Create a subset of 100 images to test on (or all if fewer than 100)
if len(imgs) > 100:
    random.seed(0) # Use a fixed seed for reproducible results
    subset = random.sample(imgs, 100)
else:
    subset = imgs
print(f"Found {len(imgs)} images, testing on {len(subset)} images.")

# --- 3. Load Model ---
model = YOLO(model_best_path)

# --- 4. Run Speed Test ---
# Run prediction once to "warm up" the GPU
print("Warming up GPU...")
model.predict(source=subset[0], imgsz=1024, verbose=False)

print("Starting speed test...")
t0 = time.time()
# Run inference on the full subset
pred = model.predict(
    source=subset,
    imgsz=1024,
    verbose=False,
    device=0
)
dt = (time.time() - t0)

# --- 5. Calculate and Print Results ---
ms_per_img = (dt / len(subset)) * 1000
fps = 1000 / ms_per_img

print("\n--- Speed Test Complete ---")
print(f"Processed {len(subset)} images in {dt:.3f} seconds.")
print(f"Average speed: {ms_per_img:.1f} ms/img")
print(f"Frames Per Second (FPS): {fps:.1f}")

Loading model for speed test: /content/drive/MyDrive/Courses/פרוייקט/colab/runs/det_pieces_a100_v1_polish/weights/best.pt
Using validation images from: /content/yolo_det/images/val
Found 330 images, testing on 100 images.
Warming up GPU...
Starting speed test...

--- Speed Test Complete ---
Processed 100 images in 27.245 seconds.
Average speed: 272.5 ms/img
Frames Per Second (FPS): 3.7


run on the test set

In [ ]:
from ultralytics import YOLO
import os

# --- 1. Define Paths ---
project_path = "/content/drive/MyDrive/Courses/פרוייקט/colab/runs"
data_yaml_path = "/content/yolo_det/data.yaml"

# --- 2. Load your BEST polished model ---
model_name = "det_pieces_a100_v1_polish"
model_path = os.path.join(project_path, model_name, "weights/best.pt")

print(f"Loading FINAL piece detection model for testing: {model_path}")

model = YOLO(model_path)

# --- 3. Run evaluation on the TEST set ---
print("\n--- Running on Piece Detection TEST Set ---")
metrics_test = model.val(
    data=data_yaml_path,
    imgsz=1024,
    batch=32,
    device=0,
    name=f"FINAL_TEST_{model_name}",
    split='test'  # <-- This is the important part
)

print("\n--- Piece Detection FINAL TEST Results ---")
print(f"mAP50-95 (Final Score): {metrics_test.box.map:.4f}")

Loading FINAL piece detection model for testing: /content/drive/MyDrive/Courses/פרוייקט/colab/runs/det_pieces_a100_v1_polish/weights/best.pt

--- Running on Piece Detection TEST Set ---
Ultralytics 8.3.221 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 92 layers, 25,846,708 parameters, 0 gradients, 78.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4337.0±675.3 MB/s, size: 1941.7 KB)
val: Scanning /content/yolo_det/labels/test.cache... 306 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 306/306 631.3Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 10/10 0.9it/s 11.6s
                   all        306       6294      0.997      0.998      0.995      0.889
            white-king        304       1624          1      0.996      0.995      0.854
           white-queen        271        433          1      0.996      0.995      0.864
            white-rook